`Done - Initiatives, keeping only scope counters`

Note that this dataset has already been converted to firm_year, the only missing part is the detail on which scopes the incentives apply. To this end, I will just keep the three relevant columns, groupby id and year, and then sum.

In [43]:
import pandas as pd
import numpy as np
import seaborn as sns

In [44]:
data = pd.read_stata("../../data/CDP Cleaning/cleaned outputs/cdp_initiative.dta")

In [45]:
data.shape

(113003, 27)

In [46]:
data.head()

,id,year,cdp_ini_type_detail_f,cdp_ini_type_f,cdp_ini_co2_c,cdp_ini_ghgscope_f,cdp_ini_disclosure_f,cdp_ini_monetarysaving_c,cdp_ini_investmentrequired_c,cdp_ini_payback_f,...,scope3,cdp_ini_subtype_f,reporting_boundary,companyname,country,industry14,isin,ticker,industry46,cdpdate
0,44,2011.0,• we have requested data from portfolio compan...,company policy or behavioral change,NaN,,,NaN,NaN,,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,NaT
1,44,2013.0,other,other,901.50,,,9303.00,1550.5,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2013-06-18
2,44,2013.0,3i are undertaking an accreditation exercise w...,energy efficiency in production processes,901.50,,,9303.00,1550.5,4-10 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2013-06-18
3,44,2014.0,reducing travel associated carbon emissions. ...,transportation,39.67,,,105766.11,0.0,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2014-06-26
4,44,2014.0,reducing electricty use with our buildings. ...,energy efficiency in buildings,10.31,,,3505.33,0.0,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2014-06-26


In [47]:
data.columns

Index(['id', 'year', 'cdp_ini_type_detail_f', 'cdp_ini_type_f',
       'cdp_ini_co2_c', 'cdp_ini_ghgscope_f', 'cdp_ini_disclosure_f',
       'cdp_ini_monetarysaving_c', 'cdp_ini_investmentrequired_c',
       'cdp_ini_payback_f', 'cdp_ini_lifetime_f', 'cdp_ini_comment_s',
       'cdp_ini_comment_lower_s', 'cdp_currency', 'currency_ex', 'scope1',
       'scope2', 'scope3', 'cdp_ini_subtype_f', 'reporting_boundary',
       'companyname', 'country', 'industry14', 'isin', 'ticker', 'industry46',
       'cdpdate'],
      dtype='object')

In [48]:
data.drop(columns=['reporting_boundary',
       'companyname', 'country', 'industry14', 'isin', 'ticker', 'industry46',
       'cdpdate', 'cdp_currency', 'currency_ex','cdp_ini_comment_s', 'cdp_ini_comment_lower_s', 'cdp_ini_ghgscope_f', 'cdp_ini_type_detail_f'], inplace=True)

In [49]:
data.columns

Index(['id', 'year', 'cdp_ini_type_f', 'cdp_ini_co2_c', 'cdp_ini_disclosure_f',
       'cdp_ini_monetarysaving_c', 'cdp_ini_investmentrequired_c',
       'cdp_ini_payback_f', 'cdp_ini_lifetime_f', 'scope1', 'scope2', 'scope3',
       'cdp_ini_subtype_f'],
      dtype='object')

In [50]:
data.sample(5)

,id,year,cdp_ini_type_f,cdp_ini_co2_c,cdp_ini_disclosure_f,cdp_ini_monetarysaving_c,cdp_ini_investmentrequired_c,cdp_ini_payback_f,cdp_ini_lifetime_f,scope1,scope2,scope3,cdp_ini_subtype_f
82553,23011,2017.0,energy efficiency in buildings,1.10,Voluntary,233.136,777.12,1-3 years,3-5 years,0.0,1.0,0.0,
86737,31426,2016.0,energy efficiency in buildings,18.49,Voluntary,5554.824,78606.00,11-15 years,21-30 years,0.0,1.0,0.0,
106521,840751,2021.0,question not applicable,NaN,Question not applicable,NaN,NaN,Question not applicable,Question not applicable,0.0,0.0,0.0,
41542,10720,2013.0,energy efficiency in buildings,671.00,,117000.000,198000.00,1-3 years,,0.0,0.0,0.0,
66549,18415,2012.0,energy efficiency in production processes,800.00,,396992.000,2481200.00,>3 years,,0.0,0.0,0.0,


In [51]:
for column in data.columns:
    print(column + ' ' + str(len(data[column].unique())))

id 10963
year 12
cdp_ini_type_f 11
cdp_ini_co2_c 24775
cdp_ini_disclosure_f 8
cdp_ini_monetarysaving_c 47663
cdp_ini_investmentrequired_c 39445
cdp_ini_payback_f 20
cdp_ini_lifetime_f 1834
scope1 2
scope2 2
scope3 2
cdp_ini_subtype_f 49


In [52]:
data = data[['id', 'year', 'scope1', 'scope2', 'scope3']]

In [53]:
data = data.groupby(['id', 'year']).sum().reset_index()

In [59]:
data['scope3'].sort_values(ascending=False)

1050     42.0
13673    28.0
4649     24.0
9485     22.0
9486     21.0
         ... 
11767     0.0
11766     0.0
11765     0.0
11764     0.0
32205     0.0
Name: scope3, Length: 32206, dtype: float32

In [33]:
# taking the log1p of the numberical variables for the scope counts
data['scope1'] = np.log1p(data['scope1'])
data['scope2'] = np.log1p(data['scope2'])
data['scope3'] = np.log1p(data['scope3'])

In [34]:
data.isna().sum()

id        0
year      0
scope1    0
scope2    0
scope3    0
dtype: int64

In [35]:
data['scope3'].mean()

0.15625417

In [36]:
data.shape

(32206, 5)

In [37]:
data.columns

Index(['id', 'year', 'scope1', 'scope2', 'scope3'], dtype='object')

In [38]:
data.shape

(32206, 5)

In [39]:
# rename columns scope 1, 2, 3 to initiative_scope1, 2, 3 binary
data.rename(columns={'scope1': 'initiative_scope1', 'scope2': 'initiative_scope2', 'scope3': 'initiative_scope3'}, inplace=True)

In [41]:
data.to_csv('../../data/processed/cdp_initiative_processed.csv', index=False)